# Trabajo práctico 4 - Balance

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

A partir del patrón de la cara de Messi (el cual debe ser seleccionado de entre las imágenes) encontrarlo en cada una de las imágenes en las que Messi aparece. Se pide un algoritmo automático que frente a la lectura sucesiva de estas imágenes (y si quieren, pueden probar con imágenes que encuentren ustedes) devuelva la posición de la cara de Messi dentro de la imagen y el nivel de confianza con el que fue hallada (deben definir también el nivel de confianza propuesto).

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.show()  